In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
lines_col_names = ['line_id', 'character_id', 'movie_id', 'character_name', 'line']
lines_df = pd.read_csv('cornell_movie_dialogs_corpus/movie_lines.txt', sep=' \+\+\+\$\+\+\+ ', engine='python', \
                header=None, names=lines_col_names)
lines_df.head()

,line_id,character_id,movie_id,character_name,line
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [3]:
movies_col_names = ['movie_id', 'movie', 'release_date', 'rating', 'votes', 'genres']
movies_df = pd.read_csv('cornell_movie_dialogs_corpus/movie_titles_metadata.txt', sep=' \+\+\+\$\+\+\+ ', \
                        engine='python', header=None, names=movies_col_names)
movies_df.head()

,movie_id,movie,release_date,rating,votes,genres
0,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
1,m1,1492: conquest of paradise,1992,6.2,10421,"['adventure', 'biography', 'drama', 'history']"
2,m2,15 minutes,2001,6.1,25854,"['action', 'crime', 'drama', 'thriller']"
3,m3,2001: a space odyssey,1968,8.4,163227,"['adventure', 'mystery', 'sci-fi']"
4,m4,48 hrs.,1982,6.9,22289,"['action', 'comedy', 'crime', 'drama', 'thrill..."


In [4]:
conversations_col_names = ['character_one_id', 'character_two_id', 'movie_id', 'conversation']
conversations_df = pd.read_csv('cornell_movie_dialogs_corpus/movie_conversations.txt', sep=' \+\+\+\$\+\+\+ ', \
                        engine='python', header=None, names=conversations_col_names)
conversations_df['conversation_id'] = ['c' + str(i) for i in range(len(conversations_df))]
conversations_df['conversation'] = conversations_df['conversation'].replace('[^a-zA-Z0-9 ]', '', regex=True)
conversations_df.head()

,character_one_id,character_two_id,movie_id,conversation,conversation_id
0,u0,u2,m0,L194 L195 L196 L197,c0
1,u0,u2,m0,L198 L199,c1
2,u0,u2,m0,L200 L201 L202 L203,c2
3,u0,u2,m0,L204 L205 L206,c3
4,u0,u2,m0,L207 L208,c4


In [5]:
split_df = conversations_df['conversation'].str.split(' ', n=None, expand=True)
split_df = pd.concat([conversations_df['conversation_id'], split_df], axis=1)
split_df = pd.melt(split_df, id_vars='conversation_id', var_name='line_pos', value_name='line_id')
split_df = split_df.sort_values(by=['conversation_id', 'line_pos'], ascending=True)
split_df.dropna(inplace=True)
split_df = split_df.reset_index(drop=True)
split_df.head()

,conversation_id,line_pos,line_id
0,c0,0,L194
1,c0,1,L195
2,c0,2,L196
3,c0,3,L197
4,c1,0,L198


In [6]:
join_df = split_df.merge(lines_df[['line_id', 'line']], on='line_id', how='left')
join_df.dropna(subset=['line'], inplace=True)
join_df['dialogue'] = join_df.groupby('conversation_id')['line'].transform(lambda x: ' '.join(x))
join_df = join_df[['conversation_id', 'dialogue']].drop_duplicates().reset_index(drop=True)
join_df.head(10)

,conversation_id,dialogue
0,c0,Can we make this quick? Roxanne Korrine and A...
1,c1,You're asking me out. That's so cute. What's ...
2,c10,How do you get your hair to look like that? Eb...
3,c100,Yeah What do you think?
4,c1000,"Hey, since when do you play Thomas Edison? Th..."
5,c10000,"Well hello there, young employee of the Sidewi..."
6,c10001,"So Josh... Look, can we talk in a minute? I'm ..."
7,c10002,"That guy rules! Who, Doug? He spends more time..."
8,c10003,Why do you even need a ride? You could walk th...
9,c10004,Aren't there a million places like this? This ...


In [7]:
lookup_df = conversations_df[['movie_id', 'conversation_id']].merge(join_df, on='conversation_id', how='left')
lookup_df

,movie_id,conversation_id,dialogue
0,m0,c0,Can we make this quick? Roxanne Korrine and A...
1,m0,c1,You're asking me out. That's so cute. What's ...
2,m0,c2,"No, no, it's my fault -- we didn't have a prop..."
3,m0,c3,Why? Unsolved mystery. She used to be really ...
4,m0,c4,"Gosh, if only we could find Kat a boyfriend......"
...,...,...,...
83092,m616,c83092,Do you think she might be interested in someo...
83093,m616,c83093,Choose your targets men. That's right Watch th...
83094,m616,c83094,Colonel Durnford... William Vereker. I hear yo...
83095,m616,c83095,"Your orders, Mr Vereker? I'm to take the Sikal..."


In [8]:
final_df = movies_df[['movie_id', 'movie', 'genres']].merge(lookup_df[['movie_id', 'dialogue']], on='movie_id',
                                                           how='left')
final_df['genres'] = final_df['genres'].replace('[^a-zA-Z,]', '', regex=True)
final_df = final_df.replace('', np.nan)
final_df.dropna(subset=['genres', 'dialogue'], inplace=True)
final_df.head()

,movie_id,movie,genres,dialogue
0,m0,10 things i hate about you,"comedy,romance",Can we make this quick? Roxanne Korrine and A...
1,m0,10 things i hate about you,"comedy,romance",You're asking me out. That's so cute. What's ...
2,m0,10 things i hate about you,"comedy,romance","No, no, it's my fault -- we didn't have a prop..."
3,m0,10 things i hate about you,"comedy,romance",Why? Unsolved mystery. She used to be really ...
4,m0,10 things i hate about you,"comedy,romance","Gosh, if only we could find Kat a boyfriend......"


In [9]:
engine = create_engine('sqlite:///dialogue.db')
final_df.to_sql('dialogue', engine, index=False, if_exists='replace')